In [12]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [13]:
def person_table(person_file_path):
        """
    Filter the population of interests and build family type for each SERIALNO
    
    Parameters:
    ----------
    person_file_path:str
        path for personal file
    
    Returns
    -------
        pd.Dataframe()
        a subset of family type and household code 
    """
    person = pd.read_csv(person_file_path,dtype={'SERIALNO': object, 'PUMA': object})
    person_sub = person[(person['AGEP']<= 64)& (person['DIS'] != 1)]
    bins = [0, 2, 5, 12, 17, 64]
    labels=['i','p','s','t','a']
    person_sub['age_regroup'] = pd.cut(person_sub['AGEP'], bins=bins, labels=labels)
    person_sub.dropna(subset=['age_regroup'], inplace = True)
    person_sub['age_regroup'] = person_sub['age_regroup'].astype(str)
    person_sub = person_sub[['SERIALNO','PUMA','age_regroup']]
    person_sub_group = person_sub.groupby(['SERIALNO','PUMA','age_regroup']).size().to_frame('count').reset_index()
    person_sub_group = person_sub_group.pivot(index='SERIALNO', columns='age_regroup', values='count')
    person_sub_group = person_sub_group.fillna(0)
    person_sub_group['c'] = person_sub_group['i']+person_sub_group['p']+person_sub_group['s']+person_sub_group['t']
    person_sub_group = person_sub_group.astype(int).astype(str)
    person_sub_group['fam_type_p'] = 'a'+person_sub_group['a']+'c'+person_sub_group['c']
    person_sub_group['fam_type'] = 'a'+person_sub_group['a']+'i'+person_sub_group['i']+'p'+person_sub_group['p']+'s'+person_sub_group['s']+'t'+person_sub_group['t']
    person_sub_group.reset_index(inplace=True)
    person_sub_group=person_sub_group[['SERIALNO','fam_type','fam_type_p']]
    return person_sub_group
    
    

In [14]:
def house_table(house_file_path,person_sub_group):
    """
    Combine household type and household income
    
    Parameters
    ----------
    house_file_path:str
        path for household file
    person_sub_group:pd.dataframe()
        return from person_table()
    
    Returns
    -------
        pd.Dataframe()
        a table with household id with their family type,  household income, puma code, state name  
    """
    house = pd.read_csv(house_file_path,dtype={'SERIALNO': object,'PUMA': object,'ST': object})
    house_sub = house[['SERIALNO','PUMA','HINCP','ST']]
    combine_family = house_sub.merge(person_sub_group,left_on = ['SERIALNO'], right_on = ['SERIALNO'], how='left')
    combine_family.dropna(subset=['HINCP','fam_type'], inplace = True)
    return combine_family
    
    

In [15]:
from sqlalchemy import create_engine, select

In [16]:
#sqlite:///Documents/Github/SSS/sss.sqlite
# "select * from puma"
def puma_sql(db_url, sql):
    """
    Combine household type and household income
    
    Parameters
    ----------
    house_file_path:str
        path for household file
    person_sub_group:pd.dataframe()
        return from person_table()
    
    Returns
    -------
        pd.Dataframe()
        a table with household id with their family type,  household income, puma code, state name  
    """
    engine = create_engine(db_url, echo=True)
    sqlite_connection = engine.connect()
    sql = sql
    puma = pd.read_sql(sql,con=engine)
    puma['place_mod'] = puma['place'].str.replace(' \(part\)','')
    puma_sub = puma[['puma_code','state_fips','place_mod','state','weight']]
    return puma_sub

In [17]:
# sqlite:///Documents/Github/SSS/sss.sqlite
# "select * from self_sufficiency_standard where year=2020"
def sss_sql(db_url, sql):
    """
    retrieve the dara from self_sufficiency_standard
    
    Parameters
    ----------
    db_url: str
        location of db
    sql: str
        syntax of sql language, like "select * from self_sufficiency_standard where year=2020"
    
    Returns
    -------
        pd.Dataframe()
        a dataframe with 'state','family_type','place','annual_self_sufficiency_wage' from db
    """
    engine = create_engine(db_url, echo=True)
    sqlite_connection = engine.connect()
    sql = sql
    sss = pd.read_sql(sql,con=engine)
    sss_sub = sss[['state','family_type','place','annual_self_sufficiency_wage']]
    return sss_sub

In [18]:
def w_avg(df, values, weights):
    """
    calculate the weight puma sss based on county standard
    
    Parameters
    ----------
    df: pd.Dataframe()
        dataframe need to be weighted
    values: 
        colunm name need to be weighted
    weights:
        colunm name that weight come from
    Returns
    -------
        float
        results of weighted number
    """
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()

In [23]:
def weight_county(sss_sub, puma_sub, combine_family,poverty_th_long):
    """
    calculate the weight puma sss based on county standard
    
    Parameters
    ----------
    sss_sub: pd.Dataframe()
        df return from sss_sql()
    puma_sub: pd.Dataframe()
        df return puma_sql()
    combine_family: pd.Dataframe()
        df return house_table()
    poverty_th_long: pd.Dataframe()
        df return poverty_th()
    Returns
    -------
        for_vis_puma: pd.DataFrame()
            % of population below poverty and sss in puma
        for_vis_state: pd.DataFrame()
            % of population below poverty and sss in puma
        comb_puma_house
            family type of each puma and their poverty line and sss
    """
    puma_county = sss_sub.merge(puma_sub,left_on = ['state','place'],right_on= ['state','place_mod'],how='left')
    puma_weight = puma_county.groupby(['state','state_fips','puma_code','family_type']).apply(w_avg, 'annual_self_sufficiency_wage', 'weight').reset_index()
    puma_weight.columns =['state', 'state_fips', 'puma_code', 'family_type', 'sss_weight']
    comb_puma_house = combine_family.merge(puma_weight,left_on=['ST','PUMA','fam_type'],right_on = ['state_fips','puma_code','family_type'],how='left')
    comb_puma_house = comb_puma_house.merge(poverty_th_long,left_on = 'fam_type_p', right_on = 'ftp', how='left')
    comb_puma_house['below'] = (comb_puma_house['HINCP']<comb_puma_house['sss_weight'])
    comb_puma_house['below_poverty'] = (comb_puma_house['HINCP']<comb_puma_house['poverty_th'])
    for_vis_puma = comb_puma_house.groupby(['state','state_fips','puma_code']).agg({'below': ['mean'],'below_poverty':['mean']}).reset_index()
    for_vis_puma.columns = ['states','state_fips','puma_code','per_below_sss_puma','per_below_poverty_puma']
    for_vis_state = comb_puma_house.groupby(['state','state_fips']).agg({'below': ['mean'],'below_poverty':['mean']}).reset_index()
    for_vis_state.columns = ['states','state_fips','per_below_sss','per_below_poverty']
    return for_vis_puma, for_vis_state, comb_puma_house
    
    
    

In [24]:
def poverty_th(poverty_th_path):
    """
    create family type and their poverty line
    
    Parameters
    ----------
    poverty_th_path: str
        location of poverty line
    Returns
    -------
        poverty_th_long: pd.DataFrame()
    """
    poverty_th = pd.read_csv(poverty_th_path)
    poverty_th_long = pd.melt(poverty_th, id_vars='total', value_vars=['c0', 'c1', 'c2','c3','c4','c5','c6','c7','c8'])
    poverty_th_long['a'] = poverty_th_long['total'].str[1].astype(int) - poverty_th_long['variable'].str[1].astype(int)
    poverty_th_long['ftp'] = 'a'+poverty_th_long['a'].astype(str)+poverty_th_long['variable']
    poverty_th_long.dropna(inplace=True)
    poverty_th_long['poverty_th'] = poverty_th_long['value'].str.replace(',','').astype(int)
    poverty_th_long = poverty_th_long[['ftp','poverty_th']]
    return poverty_th_long

In [25]:
cd ~

/Users/chengren


In [63]:
person_sub_group = person_table('Downloads/pums_us/csv_pus/psam_pusd.csv')
combine_family = house_table('Downloads/pums_us/csv_hus/psam_husd.csv',person_sub_group)


/var/folders/9y/0_t4dgms057d2y8rwm399t3w0000gp/T/ipykernel_75738/1320898156.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_sub['age_regroup'] = pd.cut(person_sub['AGEP'], bins=bins, labels=labels)
/var/folders/9y/0_t4dgms057d2y8rwm399t3w0000gp/T/ipykernel_75738/1320898156.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person_sub.dropna(subset=['age_regroup'], inplace = True)
/var/folders/9y/0_t4dgms057d2y8rwm399t3w0000gp/T/ipykernel_75738/1320898156.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [47]:
poverty_th_long = poverty_th('Downloads/poverty_th.csv')

In [27]:
puma_sub = puma_sql('sqlite:///Documents/Github/SSS/sss.sqlite',"select * from puma")
sss_sub = sss_sql('sqlite:///Documents/Github/SSS/sss.sqlite',"select * from self_sufficiency_standard where year=2020")

2022-08-12 17:02:30,330 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select * from puma")
2022-08-12 17:02:30,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-12 17:02:30,336 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select * from puma")
2022-08-12 17:02:30,337 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-12 17:02:30,338 INFO sqlalchemy.engine.Engine select * from puma
2022-08-12 17:02:30,339 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-12 17:02:31,098 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("select * from self_sufficiency_standard where year=2020")
2022-08-12 17:02:31,098 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-12 17:02:31,099 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("select * from self_sufficiency_standard where year=2020")
2022-08-12 17:02:31,099 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-08-12 17:02:31,100 INFO sqlalchemy.engine.Engine select * from self_sufficiency_standard where year=2020
2022-08-12 1

/var/folders/9y/0_t4dgms057d2y8rwm399t3w0000gp/T/ipykernel_75738/524412221.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  puma['place_mod'] = puma['place'].str.replace(' \(part\)','')


In [64]:
for_vis_puma, for_vis_state,comb_puma_house = weight_county(sss_sub, puma_sub, combine_family,poverty_th_long)

In [65]:
for_vis_puma.head()

,states,state_fips,puma_code,per_below_sss_puma,per_below_poverty_puma
0,PA,42,00101,0.341528,0.172081
1,PA,42,00102,0.159012,0.063258
2,PA,42,00200,0.190355,0.080584
3,PA,42,00300,0.182219,0.088129
4,PA,42,00400,0.203758,0.076774


In [66]:
for_vis_state.head()

,states,state_fips,per_below_sss,per_below_poverty
0,PA,42,0.189396,0.073788
1,SC,45,0.245634,0.109366
2,TN,47,0.212993,0.102598
3,TX,48,0.233565,0.101623
4,VA,51,0.196548,0.066811


In [67]:
comb_puma_house.head(10)

,SERIALNO,PUMA,HINCP,ST,fam_type,fam_type_p,state,state_fips,puma_code,family_type,sss_weight,ftp,poverty_th,below,below_poverty
0,2016000000022,01502,152900.0,42,a3i0p0s0t0,a3c0,PA,42,01502,a3i0p0s0t0,38549.210000,a3c0,21196.0,False,False
1,2016000000040,02600,8800.0,42,a1i0p0s1t0,a1c1,PA,42,02600,a1i0p0s1t0,40012.540000,a1c1,18677.0,True,True
2,2016000000065,01400,45000.0,42,a2i0p0s3t0,a2c3,PA,42,01400,a2i0p0s3t0,70302.230000,a2c3,32338.0,True,False
3,2016000000241,03800,36000.0,42,a1i0p0s0t0,a1c0,PA,42,03800,a1i0p0s0t0,20490.063500,a1c0,14097.0,False,False
4,2016000000248,02100,55000.0,42,a1i0p0s0t0,a1c0,PA,42,02100,a1i0p0s0t0,20004.350000,a1c0,14097.0,False,False
5,2016000000268,01501,120000.0,42,a1i0p0s0t0,a1c0,PA,42,01501,a1i0p0s0t0,20059.689166,a1c0,14097.0,False,False
6,2016000000272,03205,60000.0,42,a2i0p0s0t1,a2c1,PA,42,03205,a2i0p0s0t1,39833.750000,a2c1,21811.0,False,False
7,2016000000290,02200,105030.0,42,a2i0p1s0t1,a2c2,PA,42,02200,a2i0p1s0t1,45424.936046,a2c2,27479.0,False,False
8,2016000000420,03102,60000.0,42,a1i0p0s0t0,a1c0,PA,42,03102,a1i0p0s0t0,30488.370000,a1c0,14097.0,False,False
9,2016000000625,03304,85700.0,42,a3i0p0s0t0,a3c0,PA,42,03304,a3i0p0s0t0,53029.840000,a3c0,21196.0,False,False


In [68]:
for_vis_state.to_csv('for_vis_stated.csv',index=False)

In [69]:
for_vis_puma.to_csv('for_vis_pumad.csv',index=False)

In [70]:
comb_puma_house.to_csv('comb_puma_housed.csv', index=False)